In [1]:
%load_ext autoreload
%autoreload 2 

In [2]:
import numpy as np
import crocoddyl
import torch
from fnet import FNet
import torch
import torch.optim as optim
from tqdm import tqdm
import matplotlib.pyplot as plt
%matplotlib inline   

def circular(r=[2], n=[100]):
    """
    @params:
        r = list of radius
        n = list of points required from each radius
        
    @returns:
        array of points from the circumference of circle of radius r centered on origin
        
    Usage: circle_points([2, 1, 3], [100, 20, 40])
    
    """
    circles = []
    for r, n in zip(r, n):
        t = np.linspace(0, 2* np.pi, n)
        x = r * np.cos(t)
        y = r * np.sin(t)
        z = np.zeros(x.size,)
        circles.append(np.c_[x, y, z])
    return np.array(circles).squeeze()


def plot_trajectories(cost, trajectories, name = "Cost", savename=None, title=None ):
    """
    
    @params:
        cost           = list of keys for cmap
        trajectories   = list of corresponding trajectories
        name           = str, to distinguish between cost and iterations
        
    @ returns plot of trajectories colored according to keys.    
    
    """

    import matplotlib as mpl
    import matplotlib.pyplot as plt
    mpl.rcParams['figure.dpi'] = 80
    fig = plt.figure(figsize=(8, 6))

    norm = mpl.colors.Normalize(vmin=min(cost), vmax=max(cost))
    cmap = mpl.cm.ScalarMappable(norm = norm, cmap=mpl.cm.plasma)
    cmap.set_array([])


    for key, trajectory in zip(cost, trajectories):
        plt.scatter(trajectory[:, 0], trajectory[:, 1], 
                    marker = '',
                    zorder=2, 
                    s=50,
                    linewidths=0.2,
                    alpha=.8, 
                    cmap = cmap )
        plt.plot(trajectory[:, 0], trajectory[:, 1], c=cmap.to_rgba(key))

    plt.xlabel("X Coordinates", fontsize = 20)
    plt.ylabel("Y Coordinates", fontsize = 20)
    if title:
        plt.title(title)
    plt.colorbar(cmap).set_label(name, labelpad=2, size=15)
    if savename is not None:
        plt.savefig(savename+".png")
    plt.show()
    
starting_positions = circular()

In [3]:
net1 = torch.load("value.pth")

In [4]:
net2 = torch.load("hessian.pth")

In [6]:

def a2m(a):
    return np.matrix(a).T


def m2a(m):
    return np.array(m).squeeze()

class TrueHessian(crocoddyl.ActionModelAbstract):
    
    def __init__(self, net1, net2):
       
        crocoddyl.ActionModelAbstract.__init__(self, crocoddyl.StateVector(3), 2, 5)
        self.net1 = net1
        self.net2 = net2
        
    def calc(self, data, x, u=None):
        
        x = torch.as_tensor(m2a(x), dtype = torch.float32).resize_(1, 3)
        
        with torch.no_grad():
            data.cost = self.net1(x).item()

    def calcDiff(self, data, x, u=None):
        
        if u is None:
            u = self.unone
        
        x0 = torch.as_tensor(m2a(x), dtype = torch.float32).resize_(1, 3)
        j = torch.autograd.functional.jacobian(self.net1, x0).numpy().squeeze()
        h = self.net2(x0).detach().numpy().squeeze().reshape(3,3)
        #print(h)

        data.Lx = a2m(j)
        data.Lxx = a2m(h)
        

In [7]:
def solve_crocoddyl(xyz):
    """
    xyz should either be a list of an array
    """
   

    model = crocoddyl.ActionModelUnicycle()
    T = 30
    model.costWeights = np.matrix([1,1]).T
    problem = crocoddyl.ShootingProblem(np.array(xyz).T, [ model ] * T, model)
    ddp = crocoddyl.SolverDDP(problem)
    ddp.solve([], [], 1000)
    return ddp


def solve_terminal(xyz):
    model = crocoddyl.ActionModelUnicycle()
    model2 = TrueHessian(valueNet, hessianNet)
    T = 30
    model.costWeights = np.matrix([1,1]).T
    problem = crocoddyl.ShootingProblem(xyz.T, [ model ] * T, model2)
    ddp = crocoddyl.SolverDDP(problem)
    ddp.solve([], [], 1000)
    return ddp


def solution_crocoddyl(xtest):
    """
    Returns position, cost, vx[0], vxx[1]
    """
    vxx = []
    for state in xtest:        
        ddp = solve_crocoddyl(state)
        vxx_ = np.array(ddp.Vxx)
        h = vxx_[0]
        vxx.append(np.diagonal(h)[1])
    return vxx
    
def solution_terminal(xtest):
    """
    Returns position, cost, vx[0], vxx[1]
    """
    vxx = []
    for state in xtest:        
        ddp = solve_terminal(state)
        vxx_ = np.array(ddp.Vxx)
        h = vxx_[0]
        vxx.append(np.diagonal(h)[1])
    return vxx

In [8]:
def terminal_crocoddyl(stop = 1e-9):
    warmstart_cost  = []
    warmstart_iters = []
    warmstart_traj  = []

    for xyz in starting_positions:
        T = 30
        model2 = crocoddyl.ActionModelUnicycle()
        terminal_model = TrueHessian(net1, net2)
        model2.costWeights = np.matrix([1,1]).T
        problem2 = crocoddyl.ShootingProblem(m2a(xyz).T, [ model2 ] * T, terminal_model)
        ddp2 = crocoddyl.SolverFDDP(problem2)
        ddp2.th_stop= stop
        ddp2.solve([] , [], 1000)
        warmstart_cost.append(ddp2.cost)
        warmstart_iters.append(ddp2.iter)
        xs2_ = np.array(ddp2.xs)
        xs2 = xs2_[:,0:2]
        warmstart_traj.append(xs2)
        
    return warmstart_iters, warmstart_traj

In [ ]:
iters, traj = terminal_crocoddyl()

In [ ]:
plot_trajectories(iters, traj,
                  title=" Terminal crocoddyl, ddp.th_stop = 1e-9 Max_iters = 20",
                  name = "iterations")


In [ ]:
# x can be set to any constant between -2, 2
x = 1.
y = np.linspace(-1., 1., 100)
xtest = np.array([ [x,x2, 0.] for x2 in y ])

In [ ]:
vxx = solution_crocoddyl(xtest)
diff2 = solution_terminal(xtest)
vxx = np.array(vxx)
diff2 = np.array(diff2)

In [ ]:
plt.clf()
plt.figure(figsize=(10, 8))
plt.plot(xtest[:,1], vxx, '-b', label='Crocoddyl')
plt.plot(xtest[:,1], diff2,'-g', label='Crocoddyl with terminal Net')
plt.title(f"x = {x}, with theta = 0")
plt.legend()


In [ ]:



def solver_norms(xtest):
    solutions = []
    for xyz in xtest:
        ddp = solve_crocoddyl(xyz)
        solutions.append(ddp)


    vx    = []
    vxx   = []
    cost  = []


    for ddp in solutions:
        nodes  = np.array(ddp.xs)

        diff1  = np.array(ddp.Vx)
        diff2  = np.array(ddp.Vxx)

        vx.append([np.linalg.norm(diff1[0])])
        vxx.append([np.linalg.norm(diff2[0])])

        cost.append([ddp.cost])



    vx    = np.array(vx).reshape(-1, 1)
    vxx   = np.array(vxx).reshape(-1, 1)
    cost  = np.array(cost).reshape(-1,1)
    return cost, vx, vxx

def solver_grads(xtest):
    """
    Returns position, cost, vx[0], vxx[1]
    """


    vxx = []
    
    for state in xtest:        
        ddp = solve_crocoddyl(state)
              
              
        vxx_ = np.array(ddp.Vxx)
        h    = vxx_[0]
        vxx.append(np.diagonal(h)[1])
        
    
    
    return vxx





In [ ]:
# x can be set to any constant between -2, 2
x = 1.79732234
y = np.linspace(-1., 1., 100)
xtest = np.array([ [x,x2, 0.] for x2 in y ])

In [ ]:
diff2 = []
for x in xtest:
    x = torch.tensor(x, dtype = torch.float32)
    hessianNet.eval()
    with torch.no_grad():
        h = hessianNet(x)
        h = h.detach().numpy()
        h = h.reshape(3, 3)
        diff2.append(np.diagonal(h)[1])
        


In [ ]:
grad2 = []
for x in xtest:
    x = torch.tensor(x, dtype = torch.float32)
    h = torch.autograd.functional.hessian(valueNet, x).squeeze()
    h = h.detach().numpy()
    h = h.reshape(3, 3)
    grad2.append(np.diagonal(h)[1])
    
    

In [ ]:
vxx = solver_grads(xtest)

In [ ]:
grad2 = np.array(grad2)

In [ ]:
vxx = np.array(vxx)
diff2 = np.array(diff2)

In [ ]:
plt.clf()
plt.figure(figsize=(10, 8))
plt.plot(xtest[:,1], vxx, '-b', label='Crocoddyl')
plt.plot(xtest[:,1], grad2, '--r', label=' Hessian of Value Net')
plt.plot(xtest[:,1], diff2,'-g', label='Hessian Net')
plt.title(f"x = 1.79732234, with theta = 0")
plt.legend()
plt.savefig("4.png")
